Вам нужно защитить данные клиентов страховой компании разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head(3)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


In [5]:
def first_view(df):
    """
    Функция проходится по столбцам df и считает количество пропущенных, нулевых и отрицательных значений
    """
    for column in df.columns:
        na = df[column].isna().sum()
        #В столбцах могут быть текстовые значения, поэтому используем try except
        try: 
            zero = df[df[column] == 0][column].count()
            negative = df[df[column] < 0][column].count()
        except TypeError:
            zero = 0
            negative = 0
            
        
        if na>0 or zero>0 or negative>0:
            print('********В столбце', column, 'обнаружено: ********')
        if na > 0:
            print(na, 'пропущеных значений')
            print()
        if zero > 0:
            print(zero, 'нулевых значений.')
            print()
        if negative >0:
            print(negative, 'отрицательных значений')
            print()

In [6]:
first_view(data)

********В столбце Пол обнаружено: ********
2505 нулевых значений.

********В столбце Члены семьи обнаружено: ********
1513 нулевых значений.

********В столбце Страховые выплаты обнаружено: ********
4436 нулевых значений.



In [7]:
data['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4], dtype=int64)

#### Вывод по шагу 1:
    - данные загружены
    - пропуски в данных отсутствуют
    - выбивающихся значений признаков нет

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножениии матрицы признаков на обратимую матрицу предсказания **не изменятся**.

**Обоснование:** Умножим матрицу признаков на обратимую матрицу и запишем формулу обучения:

$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$

Для вычисления предсказаний умножим матрицу весов на измененную матрицу признаков

$$
a = XPw
$$
$$
a = XP((XP)^T XP)^{-1} (XP)^T y = X P(P^T X^T XP)^{-1} P^T X^T y =  
$$
$$
XP P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = XP P^{-1}w
$$


$$
X (X^T X)^{-1} X^T y
$$

Матрица весов линейной регрессии для измененных данных принимает вид:
$$
w_{преобразованная} = P^{-1}w
$$

Умножение каждого вектора признаков на одинаковую обратимую матрицу есть перевод вектора в новый базис, при этом меняется точка отсчета, но не положение вершины вектора в пространстве. При этом если вектор весов определяет гиперплоскость, проходящую с минимальным MSE (наиболее близко) к вершинам векторов признаков, то изменятся только координаты вектора весов в новом базисе а положение в пространстве будут неизменны, следовательно и предсказания модели на признаках в новом базисе будут неизменны.

**Вывод:** умножение матрицы преобразоания приводит к тому что меняются значения признаков и матрица весов линейной регрессии, но при их перемножении во время предсказаний матрица преобразования сокращается.

## 3. Алгоритм преобразования

**Алгоритм**

    1. Выделяем целевые признаки и целевой признак.
    2. Разделяем выборку на тренировочную и тестовую.
    3. Определяем численные признаки для скалирования.
    4. Преобразуем признаки тренировочной и тестовой выборок в массивы NumPy.
    5. Генерируем случайную квадратную матрицу преобразований размером равным количеству признаков.
    6. Проводим проверку обратимости полученной матрицы вычислением ее определителя.
    7. Если определитель не равен нулю:
        7.1. Умножаем матрицы признаков тренировочной и тестовой выборок на матрицу преобразований.
        7.2. Проводим скалирование матрицы признаков тренировочной и тестовой выборок.
        7.3. Обучаем модель на тренировочной выборке и получаем предсказания на тестовой.    
    8. Если определитель равен нулю - генерируем новую матрицу преобразований и переходим на п.6.

**Обоснование**

Качество линейной регрессии не поменяется потому, что, как указано в п.2 матрица весов линейной регрессии, обученной на преобразованных данных равноа:
$$
w_{преобразованная} = P^{-1}w
$$

Предсказания получаются умножением матрицы весов на измененныую матрицы признаков тестовых данных

$$
a_{преобразованная} = X_{преобразованная}w_{преобразованная} = (X P) (P^{-1}w) = X w = a_{исходная}
$$



## 4. Проверка алгоритма

#### Выделим признаки и целевой признак

In [8]:
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

#### Разобъем выборку на тренировочную и тестовую

In [9]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=12345)

#### Выделим числовые признаки для скалирования

In [10]:
numeric = features.drop('Пол', axis=1).columns
numeric

Index(['Возраст', 'Зарплата', 'Члены семьи'], dtype='object')

In [11]:
def linreg(X_train , X_test, y_train, y_test):
    """
    Функция проводит скалирование признаков, обучает модель линейной регрессии
    и проводит предсказания на тествоой выборке.
    На выходе получаем величину R2_score
    """
    #Проведем скалирование числовых признаков
    scaler = StandardScaler()
    scaler.fit(X_train.loc[:,numeric])
    X_train.loc[:,numeric] = scaler.transform(X_train.loc[:,numeric])
    X_test.loc[:,numeric] = scaler.transform(X_test.loc[:,numeric])
    
    #Обучаем модель линейной регрессии
    model = LinearRegression()
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    return r2_score(y_test, predict)

#### Построим модель линейной регрессии на исходных данных и посчитаем R2

In [12]:
R2_before = linreg(features_train, features_test, target_train, target_test)
R2_before

C:\Users\bvfrc\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\bvfrc\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


0.4305278542485148

#### Преобразуем признаки тренировочной и тестовых выборок в массивы NumPy

In [13]:
X_train, X_test = features_train.values, features_test.values
print(X_train)
print(X_test)

[[ 0.         -0.23862317 -1.98861705 -1.08239912]
 [ 0.          0.70548434 -0.26239699  0.75707499]
 [ 0.          0.35144402 -0.7671397   0.75707499]
 ...
 [ 1.          1.17753809  0.48462222 -0.16266207]
 [ 0.         -1.06471724  1.02974435  2.5965491 ]
 [ 0.         -1.41875756  0.09092291 -1.08239912]]
[[ 0.00000000e+00  2.33430582e-01 -9.07844658e-02  2.59654910e+00]
 [ 0.00000000e+00  2.23965904e+00  3.23104556e-01  7.57074988e-01]
 [ 1.00000000e+00  9.41511213e-01  2.22156014e-01 -1.08239912e+00]
 ...
 [ 1.00000000e+00 -5.92663489e-01  2.62535431e-01 -1.08239912e+00]
 [ 0.00000000e+00 -2.59629565e-03  2.82725139e-01  2.59654910e+00]
 [ 1.00000000e+00  9.41511213e-01 -1.41258737e-01 -1.62662068e-01]]


In [14]:
def random_matrix(X_train, X_test, count = 100):
    """
    Функция генерирует случайную квадратную матрицу размера равного количеству признаков
    Умножает её на матрицу признаков, обучает модель линейной регрессии на преобразованных данных и 
    считает для них метрику R2.
    На выходе получаем значение метрики и матрицу преобразования
    """
    n = X_train.shape[1]
    A = np.random.RandomState().randint(100, size=(n, n))
    flag = 0
    while flag == 0:
        if np.linalg.det(A) != 0:
            A_train = X_train @ A
            A_test = X_test @ A
            A_train = pd.DataFrame(A_train, index = features_train.index, columns = features_train.columns)
            A_test = pd.DataFrame(A_test, index = features_test.index, columns = features_test.columns)
            r2 = linreg(A_train, A_test, target_train, target_test)
            flag = 1
            print('Матрица преобразований:')
            print(A)
            print('Значение метрики R2 равно:', r2)
            
    return r2, A

In [15]:
R2_after = random_matrix(X_train, X_test)[0]

Матрица преобразований:
[[ 9 91 94 66]
 [30 86 23 87]
 [93 24 77 77]
 [12 56 52 54]]
Значение метрики R2 равно: 0.4305278542485147


#### Посчитаем разницу между метриками преобразованной и исходной матриц

In [16]:
R2_before - R2_after

1.1102230246251565e-16

## Вывод: 
    значения матрик R2 для моделей обученных на исходных и преобразованных данных практически не отличаются